In [ ]:
Test04_到每個網頁爬商品資料_Shopee
Test_Shopee

In [63]:
import pandas as pd
import re, time, requests
import requests
from selenium import webdriver
from bs4 import BeautifulSoup

In [ ]:
#解析(蝦皮headers要用Googlebot)

In [64]:
def get_soup(url):
    headers = {"user-agent": "Googlebot"}
    res = requests.get(url, headers=headers)
    return BeautifulSoup(res.text, 'lxml')

In [65]:
url = 'https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal=genius&facet=12162&noCorrection=true&page=0'
get_soup(url)

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, minimum-scale=1, user-scalable=no" name="viewport"/>
<meta content="ec0755ebf5c4bdab3e8e75640fb12995a7f73059" name="shopee:git-sha"/>
<meta content="pc-v4.42.0" name="shopee:version"/>
<!-- Open search -->
<link href="./seo/opensearch/opensearch-id.xml" rel="search" type="application/opensearchdescription+xml"/>
<link href="//s-cf-id.shopeesz.com/" rel="dns-prefetch"/>
<link href="//deo.shopeemobile.com/shopee/" rel="dns-prefetch"/>
<link href="//cv.id.xiapibuy.com/" rel="dns-prefetch"/>
<link href="https://deo.shopeemobile.com/shopee/shopee-pcmall-live-sg/assets/main.9ed111c50768ac20c583.css" rel="stylesheet"/><title data-rh="true">Shopee  Indonesia | Jual Beli di Ponsel dan Online</title><meta content="Shopee  Indonesia | Jual Beli di Ponsel dan Online" data-rh="true" property="og:title"/><meta content="noindex, follow" data-rh="true" name="robots"/><meta content

In [ ]:
#搜尋網址&換頁

In [67]:
"""直接在蝦皮搜尋"""
#url = "https://id.xiapibuy.com/search?keyword={0}&page={1}"
"""在電腦配件中搜尋"""
#url = "https://id.xiapibuy.com/search?category=134&keyword={0}&page={1}"
"""直接搜尋品牌"""
url = "https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal={0}&page={1}"

def get_urls(url, query, start_page, end_page): 
    urls = []
    for page in range(start_page, end_page+1):
        urls.append(url.format(query, page))    #query帶入url的{0}、page帶入{1}
    return urls

In [68]:
get_urls(url, "genius", 1, 2)

['https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal=genius&page=1',
 'https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal=genius&page=2']

In [ ]:
# 爬取點入網址

In [69]:
def FindLinks(pages):
    linklist = []
    for page in pages:
        
        soup = get_soup(page)
        links = soup.find_all("div",class_="col-xs-2-4 shopee-search-item-result__item")
        for link in links:
            k = "https://id.xiapibuy.com/" + link.find("a").get('href')
            linklist.append(k)
    return linklist

In [70]:
pages = get_urls(url, "genius", 1, 2)
FindLinks(pages)

['https://id.xiapibuy.com//Genius-Paketan-Keyboard-Mouse-PS2-KB-C100-i.115088768.1926056292',
 'https://id.xiapibuy.com//Genius-Mouse-DX-120-USB-Original-i.70846866.1598099812',
 'https://id.xiapibuy.com//Optical-Mouse-GENIUS-Micro-Traveler-Retractable-Cable-USB-With-1000-DPI-(ORIGINAL)-i.1644940.1133897795',
 'https://id.xiapibuy.com//Keyboard-Genius-multimedia-USB-Standart-i.8287466.3628701862',
 'https://id.xiapibuy.com//Mouse-Mini-Genius-Micro-Traveller-Retrac-Retracabel-USB-Tarik-Ulur-Mikro-Traveler-i.40715710.1583326923',
 'https://id.xiapibuy.com//Sale-Keyboard-Genius-Kb-C-100-Tidak-Dapat-Mos-i.48605435.5134625314',
 'https://id.xiapibuy.com//Mouse-Genius-DX-125-USB-i.29280008.2374156532',
 'https://id.xiapibuy.com//GENIUS-NX-7000-MOUSE-WIRELLESS-ORIGINAL-GENIUS-i.10263233.1521809243',
 'https://id.xiapibuy.com//Pen-Android-Genius-B200-Original-i.70846866.1372838978',
 'https://id.xiapibuy.com//Genius-Mouse-Pen-I608x-Pen-Tablet-i.60901488.991388022',
 'https://id.xiapibuy.com//M

In [9]:
#不含換頁版
def FindLinks(url):
    linklist = []
    soup = get_soup(url)
    links = soup.find_all("div",class_="col-xs-2-4 shopee-search-item-result__item")
    for link in links:
        k = "https://id.xiapibuy.com/" + link.find("a").get('href')
        linklist.append(k)
    return linklist

In [ ]:
# 爬取點入分頁資料

In [71]:
def get_goods(url):
    goods = []
    rows = get_soup(url)

    for row in rows:

        try:
            name = row.find('div', class_="qaNIZv").text
        except:
            name = None

        try:
            #price = row.find('div', class_="_3n5NQx").text.replace("Rp","")
            price = row.select_one("._3n5NQx").get_text().replace("Rp","")
        except:
            price = None
            
        try:
            Original_price = row.find("div", class_="_3_ISdg").text.replace("Rp","")
        except:
            Original_price = None
            
        try:
            star = row.find('div', '_3Oj5_n _2z6cUg').text
        except:
            star = None

        try:
            #reviews = row.select("._3Oj5_n")[1].get_text()
            reviews = row.find_all('div', '_3Oj5_n')[1].text
        except:
            reviews = None
            
        try:
            sold = row.find('div', '_22sp0A').text
        except:
            sold = None
            
        try:
            #stock = row.select("._1FzU2Y .items-center div+ div")[0].get_text()
            stock = row.find("div", "flex items-center _1FzU2Y").get_text()
        except:
            stock = None
        
        try:
            seller = row.select_one("._3Lybjn").get_text()
        except:
            seller = None
            
        try:
            seller_link = "https://id.xiapibuy.com" + row.find("a", "btn btn-light btn--s btn--inline btn-light--link Ed2lAD").get('href')
        except:
            seller_link = None
            
        try:
            seller_from = row.select(".kIo6pj")[-1].div.get_text()
            #seller_from = row.find_all("kIo6pj")[-1].text
        except:
            seller_from = None
            
        try:
            category = row.select_one(".kIo6pj").get_text().replace("Kategori", "")
        except:
            category = None

        try:
            brand = row.select_one("._2H-513").get_text()
        except:
            brand = None

        try:
            description = row.find("div", "_2u0jt9").get_text()
        except:
            description = None
            
        good= [name, price, Original_price, star, reviews, sold, stock, seller, seller_link, seller_from, category, brand, description]
        goods.append(good)
        
    return goods[1]  #因為不知為何第[0]列都會出現一排None，只好取第[1]列

In [ ]:
### 測試點入分頁有無爬到

In [76]:
url = 'https://id.xiapibuy.com/Stock-Promo-Mouse-USB-Genius-120-MOUSE-KABEL-GENIUS-120-MOUSE-WIRED-GENIUS-120-%E2%9C%B9-i.190378569.7519711129'
get_goods(url)

['Stock Promo Mouse USB  Genius 120/MOUSE KABEL GENIUS 120/MOUSE WIRED GENIUS 120 ✹',
 'Rp57.000',
 None,
 None,
 None,
 '0',
 'Kuantitastersisa 32 buah',
 'mahonisa',
 'https://id.xiapibuy.com/mahonisa',
 'KOTA JAKARTA PUSAT - GAMBIR, DKI JAKARTA, ID',
 'ShopeeKomputer & AksesorisMouse & Keyboards',
 'Genius',
 '🔆 𝐏𝐫𝐨𝐝𝐮𝐤 𝐓𝐞𝐫𝐛𝐚𝐭𝐚𝐬\nвυdayaĸan вaca deѕĸrιpѕι тerleвιн daнυlυ ѕeвelυм вerтanya\n\n\n\nMouse Genius Net Scroll 120 ( Warna hitam ) \n\nSpesifikasi:\n\n- Barang Baru 100%\n- Laser optik warna merah\n- Resolusi 800 dpi\n- Konektifitas USB, wired/kabel\n- OS support Windows ME/2003/XP/Vista/7/8, Mac\n- Kabel Tebal & Panjang\n- Desain praktis dan nyaman di tangan\n\nYUK Buruan di order \n\n🎵🅱 :\n🌺 SP.88▸ Mouse Usb  Genius 120/Mouse Kabel Genius 120/Mouse Wired Genius 120\n🌺 Pilih Variasi Produk atau Tulis Warna/Ukuran/Type di Rincian Pesanan.\n🌺 Gudang Kita ada di DKI Jakarta, Jawa Barat, Jawa Tengah, Jawa Timur & Bali.\n🌺 Untuk Pengiriman Barang dari STOCK Gudang yg READY, jd tdk se

In [77]:
url = 'https://id.xiapibuy.com/Genius-Paketan-Keyboard-Mouse-PS2-KB-C100-Conek.-USB-TO-PS2-i.115088768.1926095403'
get_goods(url)

['Star SellerGenius Paketan Keyboard Mouse PS2 KB-C100 & Conek. USB TO PS2',
 '',
 None,
 None,
 None,
 '2',
 'Kuantitastersisa 0 buah',
 'itstoreyes',
 'https://id.xiapibuy.com/itstoreyes',
 'KOTA YOGYAKARTA - JETIS, DI YOGYAKARTA, ID',
 'ShopeeKomputer & AksesorisMouse & KeyboardsKeyboard',
 'Genius',
 '- PS2 connection\n- Cable length 1.5 m\n- Support : Windows,Vista\n* Resolution(dpi) : 800 dpi\n* Connection Port : PS2\n* Color : Black\n* OS : Windows 7, 8, 8.1, 10\n* Sensor Engine : Optical\n* Cable length : 1.5m\n* Buttons : Three buttons (left, middle button with scroll wheel, right)\n* Both Handed : YES\n+ conek usb to ps2\n']

In [87]:
url = 'https://id.xiapibuy.com/Mouse-Genius-USB-Baru-Free-Mousepad-Logitech-i.8587449.54253808'
get_goods(url)

['Star SellerMouse Genius USB Baru + Free Mousepad Logitech',
 'Rp32.500',
 None,
 '4.2',
 '5',
 '33',
 'Kuantitastersisa 53 buah',
 'tyoshop',
 'https://id.xiapibuy.com/tyoshop',
 'KOTA BEKASI - BEKASI UTARA, JAWA BARAT, ID',
 'ShopeeKomputer & AksesorisMouse & KeyboardsMouse Pad',
 'Genius',
 'Mouse Genius NS-120 \nBonus MOusepad New setiap pembelian Mouse Genius\nPort USB\nDesign Bagus\nnyaman di Genggam\nKabel Tebal&Panjang\n(bukan Abal")\nGARANSI 1 BULAN\ndijamin Josss deh gan..\nsilahkan langsung Order aja\nstock ready...']

In [ ]:
# 爬取每一個點入頁面

In [72]:
def scraping(urls):
    ndf = []
    for i in FindLinks(urls):
        g = get_goods(i)
        ndf.append(g)
        time.sleep(0.5)
        df_all = pd.DataFrame(ndf,columns = ["name","price","Original_price","star","reviews","sold","stock","seller","seller_link","seller_from","category","brand","description"])
    return df_all

In [ ]:
# 開始爬蟲

In [90]:
scraping(urls)

,name,price,Original_price,star,reviews,sold,stock,seller,seller_link,seller_from,category,brand,description
0,Star Sellergenius mouse pen 1608x 6x8 graphic ...,Rp790.000,None,None,None,6,Kuantitastersisa 5 buah,emalljakarta,https://id.xiapibuy.com/emalljakarta,"KOTA JAKARTA UTARA - PADEMANGAN, DKI JAKARTA, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Genius MousePen i608x mempunyai bidang kerja y...
1,Star SellerMOUSE WIRELESS GENIUS NX-7005 ORIGINAL,Rp130.000,None,None,None,0,Kuantitastersisa 20 buah,autobarcode,https://id.xiapibuy.com/autobarcode,"KOTA SURABAYA - TAMBAKSARI, JAWA TIMUR, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Mouse Wireless GENIUS NX 7005 Blue Eye Origina...
2,Mouse genius dx125,Rp54.500,None,None,None,0,Kuantitastersisa 5 buah,rajendra1607,https://id.xiapibuy.com/rajendra1607,"KAB. SIDOARJO - WARU, JAWA TIMUR, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Mouse usb cable \nWarna hitam \nMerk genius \n...
3,Harga Termurah Genius Mousepen i608x,Rp519.800,None,None,None,0,Kuantitastersisa 1 buah,suta_89,https://id.xiapibuy.com/suta_89,"KAB. PEKALONGAN - KEDUNGWUNI, JAWA TENGAH, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Genius Mousepen I608X 6 X 8 Pen Stylus Tablet ...
4,Star SellerMouse Genius USB Baru + Free Mousep...,Rp32.500,None,4.2,5,33,Kuantitastersisa 53 buah,tyoshop,https://id.xiapibuy.com/tyoshop,"KOTA BEKASI - BEKASI UTARA, JAWA BARAT, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Mouse Genius NS-120 \nBonus MOusepad New setia...
5,MOUSE GENIUS DX-125,Rp53.000,None,None,None,0,Kuantitastersisa 20 buah,prayit_2010.,https://id.xiapibuy.com/prayit_2010.,"KOTA KEDIRI - KEDIRI KOTA, JAWA TIMUR, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,MOUSE GENIUS DX-125\n#original
6,Mouse Genius Second,Rp25.000,None,None,None,0,Kuantitastersisa 20 buah,asrulmf48,https://id.xiapibuy.com/asrulmf48,"KAB. BANDUNG BARAT - PADALARANG, JAWA BARAT, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Stok Banyak Barang Gudang
7,Mouse Genius Net Scroll 120 (Black)-Mouse Genius,Rp25.000,None,None,None,0,Kuantitastersisa 100 buah,wiwitdari,https://id.xiapibuy.com/wiwitdari,"KOTA JAKARTA BARAT - CENGKARENG, DKI JAKARTA, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Mouse Genius Net Scroll 120 ( Warna hitam )\n\...
8,MOUSE GENIUS DX125 USB,Rp52.000,None,None,None,0,Kuantitastersisa 10 buah,rumahkom.online,https://id.xiapibuy.com/rumahkom.online,"KAB. SLEMAN - DEPOK, DI YOGYAKARTA, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Mouse Genius DX-125 Dengan 1000 dpi sensor opt...
9,Mouse USB Genius NetScroll 120 Black,Rp39.900,None,5.0,1,1,Kuantitastersisa 100 buah,bellvinastore,https://id.xiapibuy.com/bellvinastore,"KOTA SURAKARTA (SOLO) - LAWEYAN, JAWA TENGAH, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Mouse USB Genius NetScroll 120 Black \n\nMous...


In [91]:
pd_links= pd.DataFrame(FindLinks(urls),columns = ["link"])
pd_links

,link
0,https://id.xiapibuy.com//genius-mouse-pen-1608...
1,https://id.xiapibuy.com//MOUSE-WIRELESS-GENIUS...
2,https://id.xiapibuy.com//Mouse-genius-dx125-i....
3,https://id.xiapibuy.com//Harga-Termurah-Genius...
4,https://id.xiapibuy.com//Mouse-Genius-USB-Baru...
5,https://id.xiapibuy.com//MOUSE-GENIUS-DX-125-i...
6,https://id.xiapibuy.com//Mouse-Genius-Second-i...
7,https://id.xiapibuy.com//Mouse-Genius-Net-Scro...
8,https://id.xiapibuy.com//MOUSE-GENIUS-DX125-US...
9,https://id.xiapibuy.com//Mouse-USB-Genius-NetS...


In [92]:
#測試合併
urls = 'https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal=genius&facet=12162&noCorrection=true&page=0'
pd_links = pd.DataFrame(FindLinks(urls),columns = ["link"])
scraping(urls).join(pd_links)  

,name,price,Original_price,star,reviews,sold,stock,seller,seller_link,seller_from,category,brand,description,link
0,Star Sellergenius mouse pen 1608x 6x8 graphic ...,Rp790.000,None,None,None,6,Kuantitastersisa 5 buah,emalljakarta,https://id.xiapibuy.com/emalljakarta,"KOTA JAKARTA UTARA - PADEMANGAN, DKI JAKARTA, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Genius MousePen i608x mempunyai bidang kerja y...,https://id.xiapibuy.com//genius-mouse-pen-1608...
1,Star SellerMOUSE WIRELESS GENIUS NX-7005 ORIGINAL,Rp130.000,None,None,None,0,Kuantitastersisa 20 buah,autobarcode,https://id.xiapibuy.com/autobarcode,"KOTA SURABAYA - TAMBAKSARI, JAWA TIMUR, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Mouse Wireless GENIUS NX 7005 Blue Eye Origina...,https://id.xiapibuy.com//MOUSE-WIRELESS-GENIUS...
2,Mouse genius dx125,Rp54.500,None,None,None,0,Kuantitastersisa 5 buah,rajendra1607,https://id.xiapibuy.com/rajendra1607,"KAB. SIDOARJO - WARU, JAWA TIMUR, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Mouse usb cable \nWarna hitam \nMerk genius \n...,https://id.xiapibuy.com//Mouse-genius-dx125-i....
3,Harga Termurah Genius Mousepen i608x,Rp519.800,None,None,None,0,Kuantitastersisa 1 buah,suta_89,https://id.xiapibuy.com/suta_89,"KAB. PEKALONGAN - KEDUNGWUNI, JAWA TENGAH, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Genius Mousepen I608X 6 X 8 Pen Stylus Tablet ...,https://id.xiapibuy.com//Harga-Termurah-Genius...
4,Star SellerMouse Genius USB Baru + Free Mousep...,Rp32.500,None,4.2,5,33,Kuantitastersisa 53 buah,tyoshop,https://id.xiapibuy.com/tyoshop,"KOTA BEKASI - BEKASI UTARA, JAWA BARAT, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Mouse Genius NS-120 \nBonus MOusepad New setia...,https://id.xiapibuy.com//Mouse-Genius-USB-Baru...
5,MOUSE GENIUS DX-125,Rp53.000,None,None,None,0,Kuantitastersisa 20 buah,prayit_2010.,https://id.xiapibuy.com/prayit_2010.,"KOTA KEDIRI - KEDIRI KOTA, JAWA TIMUR, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,MOUSE GENIUS DX-125\n#original,https://id.xiapibuy.com//MOUSE-GENIUS-DX-125-i...
6,Mouse Genius Second,Rp25.000,None,None,None,0,Kuantitastersisa 20 buah,asrulmf48,https://id.xiapibuy.com/asrulmf48,"KAB. BANDUNG BARAT - PADALARANG, JAWA BARAT, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Stok Banyak Barang Gudang,https://id.xiapibuy.com//Mouse-Genius-Second-i...
7,Mouse Genius Net Scroll 120 (Black)-Mouse Genius,Rp25.000,None,None,None,0,Kuantitastersisa 100 buah,wiwitdari,https://id.xiapibuy.com/wiwitdari,"KOTA JAKARTA BARAT - CENGKARENG, DKI JAKARTA, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Mouse Genius Net Scroll 120 ( Warna hitam )\n\...,https://id.xiapibuy.com//Mouse-Genius-Net-Scro...
8,MOUSE GENIUS DX125 USB,Rp52.000,None,None,None,0,Kuantitastersisa 10 buah,rumahkom.online,https://id.xiapibuy.com/rumahkom.online,"KAB. SLEMAN - DEPOK, DI YOGYAKARTA, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Mouse Genius DX-125 Dengan 1000 dpi sensor opt...,https://id.xiapibuy.com//MOUSE-GENIUS-DX125-US...
9,Mouse USB Genius NetScroll 120 Black,Rp39.900,None,5.0,1,1,Kuantitastersisa 100 buah,bellvinastore,https://id.xiapibuy.com/bellvinastore,"KOTA SURAKARTA (SOLO) - LAWEYAN, JAWA TENGAH, ID",ShopeeKomputer & AksesorisMouse & KeyboardsMou...,Genius,Mouse USB Genius NetScroll 120 Black \n\nMous...,https://id.xiapibuy.com//Mouse-USB-Genius-NetS...


In [73]:
if __name__ == "__main__":
    urls = get_urls(url, "genius", 0, 1)
    pd_links = pd.DataFrame(FindLinks(urls),columns = ["link"])
    m = scraping(urls).join(pd_links)  #要合併分別爬的商品資料&網址list，只找到join方法(但皆要先轉成DataFrame)
    m.to_csv("Shoppe_Indonisia.csv")

In [78]:
m

,name,price,Original_price,star,reviews,sold,stock,seller,seller_link,seller_from,category,brand,description,link
0,Keyboard Genius USB,Rp90.000,None,5.0,7,31,Kuantitastersisa 9 buah,agoez.in,https://id.xiapibuy.com/agoez.in,"KOTA SURAKARTA (SOLO) - LAWEYAN, JAWA TENGAH, ID",ShopeeKomputer & AksesorisMouse & KeyboardsKey...,Genius,Keyboard Genius USB \nKeyboard Pc Genius USB \...,https://id.xiapibuy.com//Keyboard-Genius-USB-i...
1,Mouse Genius DX-120,Rp100.000,None,4.9,13,443,Kuantitastersisa 50 buah,kiong56shop,https://id.xiapibuy.com/kiong56shop,"KOTA SURABAYA - TAMBAKSARI, JAWA TIMUR, ID",ShopeeKomputer & AksesorisMouse & Keyboards,Genius,Mouse Genius DX-120 dengan kabel USB terbaik,https://id.xiapibuy.com//Mouse-Genius-DX-120-i...
2,Mouse Genius USB NetScroll 120 Optical,Rp23.500,None,4.5,97,736,Kuantitastersisa 355 buah,anhoshop,https://id.xiapibuy.com/anhoshop,"KAB. TANGERANG - TELUKNAGA, BANTEN, ID",ShopeeKomputer & AksesorisMouse & Keyboards,Genius,Mouse Genius Net Scroll 120 ( Warna hitam )\n\...,https://id.xiapibuy.com//Mouse-Genius-USB-NetS...
3,Star SellerGenius Stylus Pen for Genius i405x ...,Rp350.000,None,5.0,7,12,Kuantitastersisa 1 buah,valentinecomputer,https://id.xiapibuy.com/valentinecomputer,"KOTA SURABAYA - SIMOKERTO, JAWA TIMUR, ID",ShopeeKomputer & AksesorisMouse & KeyboardsPen...,Genius,Stylus Pen Genius untuk Produk Genius i405x da...,https://id.xiapibuy.com//Genius-Stylus-Pen-for...
4,Star SellerGenius EasyPen i405X 4 x 5.5 inch S...,Rp555.000,None,4.9,7,8,Kuantitastersisa 5 buah,infinitytechno,https://id.xiapibuy.com/infinitytechno,"KOTA JAKARTA PUSAT - SAWAH BESAR, DKI JAKARTA, ID",ShopeeKomputer & AksesorisMouse & KeyboardsPen...,Genius,Genius EasyPen i405X 4 x 5.5 Original Product\...,https://id.xiapibuy.com//Genius-EasyPen-i405X-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Keyboard Genius Usb KB 100,Rp100.000,None,None,None,0,Kuantitastersisa 20 buah,rsc_komputer,https://id.xiapibuy.com/rsc_komputer,"KAB. KLATEN - JOGONALAN, JAWA TENGAH, ID",ShopeeKomputer & AksesorisMouse & KeyboardsKey...,Genius,keyboard genius usb kb-100 \ngaransi 1 bulan,https://id.xiapibuy.com//Keyboard-Genius-Usb-K...
96,Mouse GENIUS DX-120 USB Optical / Mouse GENIUS...,Rp29.999,Rp67.000,None,None,0,Kuantitastersisa 160 buah,makeit_kaby,https://id.xiapibuy.com/makeit_kaby,"KOTA JAKARTA BARAT - TAMBORA, DKI JAKARTA, ID",ShopeeKomputer & AksesorisMouse & Keyboards,Genius,GENIUS DX-120 = FULL BLACK\n\nsensor optik st...,https://id.xiapibuy.com//Mouse-GENIUS-DX-120-U...
97,Keyboard Multimedia GENIUS USB Standart,Rp100.000,None,None,None,0,Kuantitastersisa 10 buah,tokosscomsolo,https://id.xiapibuy.com/tokosscomsolo,"KOTA SURAKARTA (SOLO) - LAWEYAN, JAWA TENGAH, ID",ShopeeKomputer & AksesorisMouse & KeyboardsKey...,Genius,Keyboard Multimedia GENIUS USB Standart\n\nSpe...,https://id.xiapibuy.com//Keyboard-Multimedia-G...
98,New Paket Keyboard & Mouse genius KB100 DX125,,None,5.0,1,1,Kuantitastersisa 0 buah,mzain_comp,https://id.xiapibuy.com/mzain_comp,"KOTA SURAKARTA (SOLO) - LAWEYAN, JAWA TENGAH, ID",ShopeeKomputer & AksesorisMouse & KeyboardsKey...,Genius,Deskripsi Keyboard & Mouse Genius USB\nDeskrip...,https://id.xiapibuy.com//New-Paket-Keyboard-Mo...


In [ ]:
完整Code

In [29]:
import pandas as pd
import re, time, requests
import requests
from selenium import webdriver
from bs4 import BeautifulSoup

#解析(蝦皮headers要用Googlebot)
def get_soup(url):
    headers = {"user-agent": "Googlebot"}
    res = requests.get(url, headers=headers)
    return BeautifulSoup(res.text, 'lxml')

#搜尋網址&換頁
def get_urls(url, query, start_page, end_page): 
    urls = []
    
    for page in range(start_page, end_page+1):
        urls.append(url.format(query, page))    #query帶入url的{0}、page帶入{1}
    print(urls)    
    return urls

# 依序爬取每頁點入網址
def FindLinks(pages):
    linklist = []
    for page in pages:  
        soup = get_soup(page)
        links = soup.find_all("div",class_="col-xs-2-4 shopee-search-item-result__item")
        for link in links:
            k = "https://id.xiapibuy.com/" + link.find("a").get('href')
            linklist.append(k)
    return linklist

# 爬取點入分頁資料
def get_goods(url):
    goods = []
    rows = get_soup(url)

    for row in rows:

        try:
            name = row.find('div', class_="qaNIZv").text
        except:
            name = None

        try:
            #price = row.find('div', class_="_3n5NQx").text.replace("Rp","")
            price = row.select_one("._3n5NQx").get_text().replace("Rp","")
        except:
            price = None
            
        try:
            Original_price = row.find("div", class_="_3_ISdg").text.replace("Rp","")
        except:
            Original_price = None
            
        try:
            star = row.find('div', '_3Oj5_n _2z6cUg').text
        except:
            star = None

        try:
            #reviews = row.select("._3Oj5_n")[1].get_text()
            reviews = row.find_all('div', '_3Oj5_n')[1].text
        except:
            reviews = None
            
        try:
            sold = row.find('div', '_22sp0A').text
        except:
            sold = None
            
        try:
            #stock = row.select("._1FzU2Y .items-center div+ div")[0].get_text()
            stock = row.find("div", "flex items-center _1FzU2Y").get_text()
        except:
            stock = None
        
        try:
            seller = row.select_one("._3Lybjn").get_text()
        except:
            seller = None
            
        try:
            seller_link = "https://id.xiapibuy.com" + row.find("a", "btn btn-light btn--s btn--inline btn-light--link Ed2lAD").get('href')
        except:
            seller_link = None
            
        try:
            seller_from = row.select(".kIo6pj")[-1].div.get_text()
            #seller_from = row.find_all("kIo6pj")[-1].text
        except:
            seller_from = None
            
        try:
            category = row.select_one(".kIo6pj").get_text().replace("Kategori", "")
        except:
            category = None

        try:
            brand = row.select_one("._2H-513").get_text()
        except:
            brand = None

        try:
            description = row.find("div", "_2u0jt9").get_text()
        except:
            description = None
            
        good= [name, price, Original_price, star, reviews, sold, stock, seller, seller_link, seller_from, category, brand, description]
        goods.append(good)
        
    return goods[1]  #因為不知為何第[0]列都會出現一排None，只好取第[1]列

# 將每一個點入頁面的List依序爬取
def scraping(urls):
    ndf = []
    for idx,i in enumerate(FindLinks(urls)):  #記錄目前進行的迴圈次數，配上總迴圈次數，可做為進度條使用。
        print("Crawing No." + str(idx+1) + " Item in Total:" + str(len(FindLinks(urls))) + "Item")
        
        g = get_goods(i)
        ndf.append(g)
        time.sleep(0.2)
        df_all = pd.DataFrame(ndf,columns = ["name","price","Original_price","star","reviews","sold","stock","seller","seller_link","seller_from","category","brand","description"])
    return df_all

# 開始爬蟲
if __name__ == "__main__":
    """直接在蝦皮搜尋"""
    #url = "https://id.xiapibuy.com/search?keyword={0}&page={1}"
    """在電腦配件中搜尋"""
    #url = "https://id.xiapibuy.com/search?category=134&keyword={0}&page={1}"
    """直接搜尋品牌"""
    url = "https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal={0}&page={1}"
    
    urls = get_urls(url, "hp",11 , 15)
    
    pd_links = pd.DataFrame(FindLinks(urls),columns = ["link"])
    #要合併分別爬的商品資料&網址list，只找到join方法(但皆要先轉成DataFrame)
    #且要同時跑才會對應到一樣順序的資料
    m = scraping(urls).join(pd_links)  
    m.to_csv("Shoppe_Indonisia3.csv")

['https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal=hp&page=11', 'https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal=hp&page=12', 'https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal=hp&page=13', 'https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal=hp&page=14', 'https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal=hp&page=15']
Crawing No.1 Item in Total:250Item


KeyboardInterrupt: 

### 不用pandas合併

In [18]:
import pandas as pd
import re, time, requests
import requests
from selenium import webdriver
from bs4 import BeautifulSoup

#解析(蝦皮headers要用Googlebot)
def get_soup(url):
    headers = {"user-agent": "Googlebot"}
    res = requests.get(url, headers=headers)
    return BeautifulSoup(res.text, 'lxml')

#搜尋網址&換頁
def get_urls(url, query, start_page, end_page): 
    urls = []
    
    for page in range(start_page, end_page+1):
        urls.append(url.format(query, page))    #query帶入url的{0}、page帶入{1}
    print(urls)    
    return urls

# 依序爬取每頁點入網址
def FindLinks(pages):
    linklist = []
    for page in pages:  
        soup = get_soup(page)
        links = soup.find_all("div",class_="col-xs-2-4 shopee-search-item-result__item")
        for link in links:
            k = "https://id.xiapibuy.com/" + link.find("a").get('href')
            linklist.append(k)
    return linklist

# 爬取點入分頁資料
def get_goods(url):
    goods = []
    rows = get_soup(url)

    for row in rows:

        try:
            name = row.find('div', class_="qaNIZv").text
        except:
            name = None

        try:
            #price = row.find('div', class_="_3n5NQx").text.replace("Rp","")
            price = row.select_one("._3n5NQx").get_text().replace("Rp","")
        except:
            price = None
            
        try:
            Original_price = row.find("div", class_="_3_ISdg").text.replace("Rp","")
        except:
            Original_price = None
            
        try:
            star = row.find('div', '_3Oj5_n _2z6cUg').text
        except:
            star = None

        try:
            #reviews = row.select("._3Oj5_n")[1].get_text()
            reviews = row.find_all('div', '_3Oj5_n')[1].text
        except:
            reviews = None
            
        try:
            sold = row.find('div', '_22sp0A').text
        except:
            sold = None
            
        try:
            #stock = row.select("._1FzU2Y .items-center div+ div")[0].get_text()
            stock = row.find("div", "flex items-center _1FzU2Y").get_text()
        except:
            stock = None
        
        try:
            seller = row.select_one("._3Lybjn").get_text()
        except:
            seller = None
            
        try:
            seller_link = "https://id.xiapibuy.com" + row.find("a", "btn btn-light btn--s btn--inline btn-light--link Ed2lAD").get('href')
        except:
            seller_link = None
            
        try:
            seller_from = row.select(".kIo6pj")[-1].div.get_text()
            #seller_from = row.find_all("kIo6pj")[-1].text
        except:
            seller_from = None
            
        try:
            category = row.select_one(".kIo6pj").get_text().replace("Kategori", "")
        except:
            category = None

        try:
            brand = row.select_one("._2H-513").get_text()
        except:
            brand = None

        try:
            description = row.find("div", "_2u0jt9").get_text()
        except:
            description = None
            
        good= [name, price, Original_price, star, reviews, sold, stock, seller, seller_link, seller_from, category, brand, description]
        goods.append(good)
        
    return goods[1]  #因為不知為何第[0]列都會出現一排None，只好取第[1]列

In [17]:
get_goods("https://id.xiapibuy.com/KEYBOARD-HP-Mini-210-1000-210-2000-i.12268176.107543765")

['Star SellerKEYBOARD HP Mini 210-1000, 210-2000',
 '88.000',
 None,
 '5.0',
 '1',
 '1',
 'Kuantitastersisa 4 buah',
 'pusatlaptopid',
 'https://id.xiapibuy.com/pusatlaptopid',
 'KOTA SURABAYA - GUBENG, JAWA TIMUR, ID',
 'ShopeeKomputer & AksesorisMouse & KeyboardsKeyboard',
 None,
 'KEYBOARD HP Mini 210-1000, 210-2000\n\nWarna: HITAM\nKondisi : Baru / New. \n\nGaransi 3 bulan. GAK PAKE RIBET !!!\nGambar sesuai dengan foto asli...\nKualitas ORIGINAL !!!\n\nSebelum order di mohon PM untuk menanyakan ketersediaan stock barang nya\nSelamat Berbelanja :)']

In [21]:
# 將每一個點入頁面的List依序爬取
def scraping(urls):
    all_goods = ["name","price","Original_price","star","reviews","sold","stock","seller","seller_link","seller_from","category","brand","description"]
    for idx,i in enumerate(FindLinks(urls)):  #記錄目前進行的迴圈次數，配上總迴圈次數，可做為進度條使用。
        print("Crawing No." + str(idx+1) + " Item in Total:" + str(len(FindLinks(urls))) + "Item")
        
        goods = get_goods(i)
        time.sleep(0.2)
        all_goods = all_goods + goods
    return all_goods

# 開始爬蟲
if __name__ == "__main__":
    """直接在蝦皮搜尋"""
    #url = "https://id.xiapibuy.com/search?keyword={0}&page={1}"
    """在電腦配件中搜尋"""
    #url = "https://id.xiapibuy.com/search?category=134&keyword={0}&page={1}"
    """直接搜尋品牌"""
    url = "https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal={0}&page={1}"
    
    urls = get_urls(url, "logitech",0 , 0)
    m = scraping(urls).extend(FindLinks(urls))  #extend:列表合併
    #要合併分別爬的商品資料&網址list，只找到join方法(但皆要先轉成DataFrame)
    #且要同時跑才會對應到一樣順序的資料
    pd = pd.DataFrame(m)
    pd.to_csv("Shoppe_Indonisia3.csv")

['https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal=logitech&page=0']
Crawing No.1 Item in Total:50Item
Crawing No.2 Item in Total:50Item
Crawing No.3 Item in Total:50Item
Crawing No.4 Item in Total:50Item
Crawing No.5 Item in Total:50Item
Crawing No.6 Item in Total:50Item
Crawing No.7 Item in Total:50Item
Crawing No.8 Item in Total:50Item
Crawing No.9 Item in Total:50Item
Crawing No.10 Item in Total:50Item
Crawing No.11 Item in Total:50Item
Crawing No.12 Item in Total:50Item
Crawing No.13 Item in Total:50Item
Crawing No.14 Item in Total:50Item
Crawing No.15 Item in Total:50Item
Crawing No.16 Item in Total:50Item
Crawing No.17 Item in Total:50Item
Crawing No.18 Item in Total:50Item
Crawing No.19 Item in Total:50Item
Crawing No.20 Item in Total:50Item
Crawing No.21 Item in Total:50Item
Crawing No.22 Item in Total:50Item
Crawing No.23 Item in Total:50Item
Crawing No.24 Item in Total:50Item
Crawing No.25 Item in Total:50Item
Crawing No.26 Item in Total:50Item
Crawing No.2

In [24]:
# 開始爬蟲
if __name__ == "__main__":
    """直接在蝦皮搜尋"""
    #url = "https://id.xiapibuy.com/search?keyword={0}&page={1}"
    """在電腦配件中搜尋"""
    #url = "https://id.xiapibuy.com/search?category=134&keyword={0}&page={1}"
    """直接搜尋品牌"""
    url = "https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal={0}&page={1}"
    
    urls = get_urls(url, "logitech",0 , 0)
    m = scraping(urls).extend(FindLinks(urls))  #extend:列表合併
    #要合併分別爬的商品資料&網址list，只找到join方法(但皆要先轉成DataFrame)
    #且要同時跑才會對應到一樣順序的資料
    print(m)

['https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal=logitech&page=0']
Crawing No.1 Item in Total:50Item
Crawing No.2 Item in Total:50Item
Crawing No.3 Item in Total:50Item
Crawing No.4 Item in Total:50Item
Crawing No.5 Item in Total:50Item
Crawing No.6 Item in Total:50Item
Crawing No.7 Item in Total:50Item
Crawing No.8 Item in Total:50Item
Crawing No.9 Item in Total:50Item
Crawing No.10 Item in Total:50Item
Crawing No.11 Item in Total:50Item
Crawing No.12 Item in Total:50Item
Crawing No.13 Item in Total:50Item
Crawing No.14 Item in Total:50Item
Crawing No.15 Item in Total:50Item
Crawing No.16 Item in Total:50Item
Crawing No.17 Item in Total:50Item
Crawing No.18 Item in Total:50Item
Crawing No.19 Item in Total:50Item
Crawing No.20 Item in Total:50Item
Crawing No.21 Item in Total:50Item
Crawing No.22 Item in Total:50Item
Crawing No.23 Item in Total:50Item
Crawing No.24 Item in Total:50Item
Crawing No.25 Item in Total:50Item
Crawing No.26 Item in Total:50Item
Crawing No.2